## How to implement Linear Regression

        We will look into how we could write our own implementation

In [2]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy , numpy as np
from scipy import linalg

In [3]:
np.set_printoptions(precision=6)

In [4]:
data = datasets.load_diabetes()

In [5]:
feature_names=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [6]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [7]:
trn.shape, test.shape

((353, 10), (89, 10))

In [8]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred))

## How did sklearn do it?

        By checking the source code, you can see that in the dense case, it calls scipy.linalg.lstqr , which is calling the LAPACK method:

        1. gelsd : uses SVD and a divide and conquer method
        2. gelsy : uses QR factorization
        3. gelss : uses SVD


    Scipy Sparse Least Squares
    It uses an iterative method called Golub and Kahan bidiagonalization
    Preconditioning is another way to reduce the number of iterations. If it is possible to solve a related system M*x = b, efficiently, where M approximates A in some helpful way, LSQR may converge more rapidly on the system

#### linalg.lstqr
        The sklearn implementation handeled adding a constant term for us

In [9]:
trn_int = np.c_[trn, np.ones(trn.shape[0])]
test_int = np.c_[test, np.ones(test.shape[0])]

        Since linalg.lstsq lets us specify which LAPACK routine we want to use, lets try them all and do some timing comparisons:

In [10]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelsd")

The slowest run took 5.14 times longer than the fastest. This could mean that an intermediate result is being cached.
607 µs ± 510 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelsy")

1.24 ms ± 424 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit coef, _,_,_ = linalg.lstsq(trn_int, y_trn, lapack_driver="gelss")

The slowest run took 4.58 times longer than the fastest. This could mean that an intermediate result is being cached.
587 µs ± 366 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Naive Solution
    Recall that we want to find x that minimizes:
$$ ||Ax-b||_2 $$
    Another way to think about this is that we are interested in where vector b is closest to the subspace spanned by A. This is the projection of b onto A. Since b-Ax must be perpendicular to the subspace spanned by A, we see that

$$ A^T (b - Ax) = 0 $$

    This leads us to the normal equations
$$ x = (A^T A)^{-1} A^Tb $$

In [13]:
def ls_naive(A, b):
     return np.linalg.inv(A.T @ A) @ A.T @ b

In [14]:
%timeit coeffs_naive = ls_naive(trn_int, y_trn)

The slowest run took 13.44 times longer than the fastest. This could mean that an intermediate result is being cached.
128 ms ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
coeffs_naive = ls_naive(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_naive)

(56.32960590877427, 47.82386741198034)

### Normal Equations (Cholesky)

    Normal Equations :

$$ A^T Ax = A^T b $$

If A has full rank, the pseudo-inverse $(A^TA)^{-1} A^T$ is a square, hermitian positive definite matrix. The satndard way of solving such a system is Cholesky Factorization, which dfinds iupper-triangular R, s.t $A^T A = R^TR$

In [16]:
A = trn_int

In [17]:
b = y_trn

In [18]:
AtA = A.T @ A
Atb = A.T @ b

In [19]:
R = scipy.linalg.cholesky(AtA)

In [20]:
np.set_printoptions(suppress=True, precision=4)
R

array([[ 0.912 ,  0.1804,  0.1842,  0.3173,  0.2687,  0.2217, -0.0509,
         0.1901,  0.2657,  0.3002, -0.1708],
       [ 0.    ,  0.8748,  0.0565,  0.1561, -0.0118,  0.1097, -0.3566,
         0.2886,  0.0754,  0.1325, -0.1085],
       [ 0.    ,  0.    ,  0.8607,  0.307 ,  0.169 ,  0.1812, -0.2942,
         0.2979,  0.3477,  0.3038, -0.2325],
       [ 0.    ,  0.    ,  0.    ,  0.7696,  0.1211,  0.0471,  0.0286,
         0.0233,  0.1789,  0.163 ,  0.0297],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.8123,  0.7239,  0.1359,
         0.3783,  0.33  ,  0.1165, -1.2538],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.3846, -0.4024,
         0.2558, -0.3441, -0.0579, -0.7216],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.6497,
        -0.4917, -0.5276, -0.1386,  0.3961],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.2804, -0.018 ,  0.0037, -0.3866],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
 

In [21]:
np.linalg.norm(AtA - R.T @ R)

5.684557367178372e-14

$$ A^T Ax = A^T b $$
$$ R^T Rx = A^T b $$
$$ R^T w = A^T b $$
$$ Rx = w $$

In [22]:
w = scipy.linalg.solve_triangular(R, Atb, lower=False, trans='T')

In [23]:
np.linalg.norm(R.T @ w - Atb)

7.277126723879252e-12

In [24]:
coeffs_chol = scipy.linalg.solve_triangular(R, w, lower=False)


In [25]:
np.linalg.norm(R @ coeffs_chol - w)

1.3048672769382318e-13

In [26]:
def ls_chol(A, b):
    R = scipy.linalg.cholesky(A.T @ A)
    w = scipy.linalg.solve_triangular(R, A.T @ b, trans='T')
    return scipy.linalg.solve_triangular(R, w)

In [27]:
%timeit coeffs_chol = ls_chol(trn_int, y_trn)

The slowest run took 8.18 times longer than the fastest. This could mean that an intermediate result is being cached.
120 ms ± 65.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
coeffs_chol = ls_chol(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_chol)

(56.3296059087742, 47.82386741198027)

### QR Factorization
$$ Ax = b $$

$$ A = QR$$
$$ QRx = b $$
$$ Rx = Q^T b$$

In [29]:
def ls_qr(A,b):
    Q, R = scipy.linalg.qr(A, mode='economic')
    return scipy.linalg.solve_triangular(R, Q.T @ b)

In [30]:
%timeit coeffs_qr = ls_qr(trn_int, y_trn)

409 µs ± 103 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
coeffs_qr = ls_qr(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_qr)

(56.32960590877418, 47.82386741198025)

### SVD    

$$ Ax = b $$
$$ A = U \sum V$$
$$ \sum V x = U^T b $$
$$ \sum w = U^T b $$
$$ x = V^T w $$

        SVD gives a pseudo-inverse

In [32]:
def ls_svd(A,b):
    m, n = A.shape
    U, sigma, Vh = scipy.linalg.svd(A, full_matrices=False, lapack_driver='gesdd')
    w = (U.T @ b)/ sigma
    return Vh.T @ w

In [33]:
%timeit coeffs_svd = ls_svd(trn_int, y_trn)

550 µs ± 149 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%timeit coeffs_svd = ls_svd(trn_int, y_trn)

2.44 ms ± 775 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
coeffs_svd = ls_svd(trn_int, y_trn)
regr_metrics(y_test, test_int @ coeffs_svd)

(56.32960590877416, 47.82386741198024)

### Random Sketching Technique for Least Squares Regression

        1. Sample a r x n random matrix
        2. Compute SA and Sb
        3. Find Exact Solution x to regression SA x = Sb

### Timing Comparison

In [36]:
import timeit
import pandas as pd

In [37]:
def scipylstq(A, b):
    return scipy.linalg.lstsq(A,b)[0]

In [38]:
row_names = ['Normal Eqns- Naive',
             'Normal Eqns- Cholesky', 
             'QR Factorization', 
             'SVD', 
             'Scipy lstsq']

name2func = {'Normal Eqns- Naive': 'ls_naive', 
             'Normal Eqns- Cholesky': 'ls_chol', 
             'QR Factorization': 'ls_qr',
             'SVD': 'ls_svd',
             'Scipy lstsq': 'scipylstq'}

In [39]:
m_array = np.array([100, 1000, 10000])
n_array = np.array([20, 100, 1000])

In [40]:
index = pd.MultiIndex.from_product([m_array, n_array], names=['# rows', '# cols'])
pd.options.display.float_format = '{:,.6f}'.format
df = pd.DataFrame(index=row_names, columns=index)
df_error = pd.DataFrame(index=row_names, columns=index)

In [ ]:
# %%prun
for m in m_array:
    for n in n_array:
        if m >= n:        
            x = np.random.uniform(-10,10,n)
            A = np.random.uniform(-40,40,[m,n])   # removed np.asfortranarray
            b = np.matmul(A, x) + np.random.normal(0,2,m)
            for name in row_names:
                fcn = name2func[name]
                t = timeit.timeit(fcn + '(A,b)', number=5, globals=globals())
                df.set_value(name, (m,n), t)
                coeffs = locals()[fcn](A, b)
                reg_met = regr_metrics(b, A @ coeffs)
                df_error.set_value(name, (m,n), reg_met[0])

In [ ]:
df

In [ ]:
df_error

In [ ]:
store = pd.HDFStore('least_squares_results.h5')

In [ ]:
store['df'] = df

### Conditioning & Stability

        Condition Number is a measure of how small changes to the input cause the output to change.The relative condition number is defined by 

$$ \kappa = \text{sup} \frac{||\delta f ||}{|| f(x)||} / \frac{||\delta x||}{||x||} $$


        Conditioning : perturbation behavior of a mathematical problem
        Stability : perturabation  behavior of an algorithm used to solve that problem on a computer


### Conditioning Example
        The problem of computing eigenvalues of a non-symmetric matrix is often ill-conditioned

In [48]:
A = [[1, 1000], [0, 1]]
B = [[1, 1000], [0.001, 1]]

In [49]:
wA, vrA = scipy.linalg.eig(A)
wB, vrB = scipy.linalg.eig(B)

In [50]:
wA, wB

(array([1.+0.j, 1.+0.j]), array([2.+0.j, 0.+0.j]))

### Condition Number of a Matrix

The product $||A|| || A^{-1}|| $ come up so often it has its own name: the condition number of A. Note that normalyy we talk about the conditioning of problems, not matrices.

        The condition number of A relates to :
        1. computing b given A and x in Ax = b
        2. computing x given A and b in Ax = b

### Matrix Inversion is Unstable

In [51]:
from scipy.linalg import hilbert

In [52]:
n = 14
A = hilbert(n)
x = np.random.uniform(-10,10,n)
b = A @ x

In [53]:
A_inv = np.linalg.inv(A)

In [54]:
np.linalg.norm(np.eye(n) - A @ A_inv)

4.7177114010624965

In [55]:
np.linalg.cond(A)

9.274324459181185e+17

In [56]:
A @ A_inv

array([[ 1.    ,  0.    ,  0.    ,  0.0003, -0.0008, -0.0033,  0.0068,
        -0.0587, -0.0236,  0.3125, -0.9121, -0.1484,  0.0065, -0.0569],
       [-0.    ,  1.    , -0.0001,  0.0014, -0.0126,  0.0785, -0.1674,
         0.2113, -0.2763,  0.625 , -0.9451, -0.1367,  0.1234, -0.0101],
       [ 0.    , -0.    ,  1.    , -0.0002,  0.002 ,  0.002 ,  0.0156,
        -0.0156,  0.3125,  0.125 , -0.2812, -0.0312,  0.    , -0.0127],
       [ 0.    ,  0.    ,  0.    ,  0.9999,  0.0047, -0.0407,  0.0894,
        -0.6306,  0.9308, -1.4375,  1.2176, -1.0351,  0.4388, -0.0656],
       [ 0.    , -0.    ,  0.    , -0.    ,  1.0011,  0.0031,  0.0148,
        -0.1498,  0.4747,  0.    ,  0.2246, -0.0547,  0.0454, -0.0039],
       [ 0.    , -0.    ,  0.    , -0.0002,  0.0062,  0.964 ,  0.1136,
        -0.3853,  0.6504, -0.4688,  0.5615, -0.2886,  0.2377, -0.0267],
       [ 0.    , -0.    ,  0.    , -0.0005,  0.007 , -0.0264,  1.0804,
        -0.2277,  0.6103, -0.2188,  0.1478, -0.1211,  0.0712, -0.0013],

In [57]:
row_names = ['Normal Eqns- Naive',
             'QR Factorization', 
             'SVD', 
             'Scipy lstsq']

name2func = {'Normal Eqns- Naive': 'ls_naive', 
             'QR Factorization': 'ls_qr',
             'SVD': 'ls_svd',
             'Scipy lstsq': 'scipylstq'}

In [58]:
pd.options.display.float_format = '{:,.9f}'.format
df = pd.DataFrame(index=row_names, columns=['Time', 'Error'])

        Even if A is incredibly sparse, A inverse is generally dense. For large matrices , A inverse could be so dense as to not fit in memory


## Runtime
1. Matrix Inversion : $2n^3$
2. Matrix Multiplication : $n^3$
3. Cholesky : $\frac{1}{3} n^3$
4. QR, Gram Schmidt : $ 2mn^2$, $m \ge n$
5. QR, Householder : $2mn^2 - \frac{2}{3} n^3$
6. Solving a triangular system : $n^2$